In [1]:
import shutil
import apiquery
import pandas as pd
import seaborn as sns
import os
import numpy as np
import random
DATA_PATH = '../01.Data'
shutil.copy("apiquery_pyc.py", "apiquery.pyc")
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
import math
from collections import Counter
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import math
import time
!pip install lightgbm
import lightgbm as lgbm
import matplotlib.pyplot as plt

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [2]:
%%time
df_train = pd.read_csv(os.path.join("../01.Data",'fold.csv'))
df_test      = pd.read_csv(os.path.join(DATA_PATH,'test_preprocessed.csv'))
y_submission = pd.read_csv(os.path.join(DATA_PATH,'y_test_submission_example.tsv'), index_col='Index', encoding='utf-8', sep='\t')
df_real_test = pd.read_csv(os.path.join(DATA_PATH,'test_real_preprocessed.csv'))
df_train.head(1)

CPU times: user 3.01 s, sys: 383 ms, total: 3.4 s
Wall time: 3.4 s


,creator,product_name,quantity,packaging,packaging_tags,brands,brands_tags,categories_tags,origins_tags,manufacturing_places_tags,labels_tags,labels_en,countries_tags,ingredients_text,allergens,traces_tags,serving_size,additives_n,additives,additives_tags,ingredients_from_palm_oil_n,ingredients_that_may_be_from_palm_oil_n,pnns_groups_1,pnns_groups_2,states_en,target,created_datetime_year,created_datetime_month,created_datetime_day,created_datetime_hour,created_datetime_minute,created_datetime_second,last_modified_datetime_year,last_modified_datetime_month,last_modified_datetime_day,last_modified_datetime_hour,last_modified_datetime_minute,last_modified_datetime_second,main_category_lang,main_category_cat,main_category_en_lang,main_category_en_cat,states_en_brands,states_en_categories,states_en_characteristics,states_en_expiration date,states_en_general_check,states_en_general_complete,states_en_ingredients,states_en_nutrition facts,states_en_packaging,states_en_packaging-code-,states_en_photo_upload,states_en_photo_validate,states_en_product name,states_en_quantity,diff_t,target_bin,fold
0,usda-ndb-import,caramel creams,NaN,NaN,NaN,gold emblem,gold emblem,NaN,NaN,NaN,NaN,NaN,en:united-states,"Wheat flour, corn syrup, dextrose, sugar, part...",NaN,NaN,34 g (3 PIECES),1.0,[ wheat-flour -> en:wheat-flour ] [ flour -...,"en:e322,en:e322i",0.0,0.0,NaN,NaN,"To be completed,Nutrition facts completed,Ingr...",11.0,2017,3,10,8,46,35,2017,3,10,8,46,35,NaN,NaN,NaN,NaN,completed,to be completed,to be completed,to be completed,NaN,to be completed,completed,completed,to be completed,to be completed,photos to be uploaded,NaN,completed,to be completed,0,8,2


In [3]:
df_test.head()

,creator,product_name,quantity,packaging,packaging_tags,brands,brands_tags,categories_tags,origins_tags,manufacturing_places_tags,labels_tags,labels_en,countries_tags,ingredients_text,allergens,traces_tags,serving_size,additives_n,additives,additives_tags,ingredients_from_palm_oil_n,ingredients_that_may_be_from_palm_oil_n,pnns_groups_1,pnns_groups_2,states_en,created_datetime_year,created_datetime_month,created_datetime_day,created_datetime_hour,created_datetime_minute,created_datetime_second,last_modified_datetime_year,last_modified_datetime_month,last_modified_datetime_day,last_modified_datetime_hour,last_modified_datetime_minute,last_modified_datetime_second,main_category_lang,main_category_cat,main_category_en_lang,main_category_en_cat,states_en_brands,states_en_categories,states_en_characteristics,states_en_expiration date,states_en_general_check,states_en_general_complete,states_en_ingredients,states_en_nutrition facts,states_en_packaging,states_en_packaging-code-,states_en_photo_upload,states_en_photo_validate,states_en_product name,states_en_quantity,diff_t
0,openfoodfacts-contributors,tomato ketchup,"500 ml, 580 g","flasche,kunststoff,flasche","flasche,kunststoff,flasche","mcdonald's,develey","mcdonald s,develey","en:groceries,en:sauces,en:tomato sauces,en:ket...",NaN,NaN,NaN,NaN,en:germany,"Tomatenmark* (66%), Glukose-Fruktose-Sirup, Br...",NaN,NaN,NaN,0.0,[ tomatenmark -> de:tomatenmark ] [ 66 -> d...,NaN,0.0,0.0,fat and sauces,dressings and sauces,"To be completed,Nutrition facts completed,Ingr...",2014,8,23,16,16,45,2016,7,17,7,28,24,en,groceries,NaN,Groceries,completed,completed,completed,to be completed,NaN,to be completed,completed,completed,completed,NaN,photos uploaded,photos to be validated,completed,completed,59929899
1,usda-ndb-import,grated italian style blend cheese,NaN,NaN,NaN,western family,western family,NaN,NaN,NaN,NaN,NaN,en:united-states,"Parmesan cheese (cultured part-skim milk, salt...",NaN,NaN,5 g (2 tsp),0.0,[ parmesan-cheese -> en:parmesan-cheese ] [...,NaN,0.0,0.0,NaN,NaN,"To be completed,Nutrition facts completed,Ingr...",2017,3,9,13,31,47,2017,3,9,13,31,47,NaN,NaN,NaN,NaN,completed,to be completed,to be completed,to be completed,NaN,to be completed,completed,completed,to be completed,to be completed,photos to be uploaded,NaN,completed,to be completed,0
2,usda-ndb-import,empanadas de pollo chicken turnovers,NaN,NaN,NaN,quirch,quirch,NaN,NaN,NaN,NaN,NaN,en:united-states,"Filling ingredients: chicken, water, cornstarc...",NaN,NaN,108 g (1 TURNOVER),3.0,[ filling-ingredients -> en:filling-ingredien...,"en:e330,en:e509,en:e621",0.0,0.0,NaN,NaN,"To be completed,Nutrition facts completed,Ingr...",2017,3,10,9,16,9,2017,3,10,9,16,10,NaN,NaN,NaN,NaN,completed,to be completed,to be completed,to be completed,NaN,to be completed,completed,completed,to be completed,to be completed,photos to be uploaded,NaN,completed,to be completed,1
3,usda-ndb-import,ice cream,NaN,NaN,NaN,topco associates inc.,topco associates inc,NaN,NaN,NaN,NaN,NaN,en:united-states,"Milkfat and nonfat milk, sugar, high fructose ...",NaN,NaN,65 g (0.5 cup),3.0,[ milkfat-and-nonfat-milk -> en:milkfat-and-n...,"en:e412,en:e466,en:e471",0.0,0.0,NaN,NaN,"To be completed,Nutrition facts completed,Ingr...",2017,3,9,20,16,59,2017,3,9,20,16,59,NaN,NaN,NaN,NaN,completed,to be completed,to be completed,to be completed,NaN,to be completed,completed,completed,to be completed,to be completed,photos to be uploaded,NaN,completed,to be completed,0
4,usda-ndb-import,"organic dressing, creamy caesar",NaN,NaN,NaN,"365 everyday value, whole foods market inc.","365 everyday value,whole foods market inc",NaN,NaN,NaN,NaN,NaN,en:united-states,"Organic expeller pressed soybean oil, water, o...",NaN,NaN,30 ml (2 Tbsp),0.0,[ organic-expeller-pressed-soybean-oil -> en:...,NaN,0.0,0.0,NaN,NaN,"To be completed,Nutrition facts completed,Ingr...",2017,3,9,16,31,45,2017,3,9,16,31,45,NaN,NaN,NaN,NaN,completed,to be completed,to be completed,to be completed,NaN,to be co